## PHYSICS MODEL

In [ ]:
#Importing the libraries and the datasets 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
import seaborn as sns
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
df_X_train = pd.read_csv("X_train_phy.csv")
df_X_test = pd.read_csv("X_test_phy.csv")
df_y_train = pd.read_csv("y_train_phy.csv")
df_y_test = pd.read_csv("y_test_phy.csv")
df_y_test = df_y_test.astype(int)
df_y_train = df_y_train.astype(int)

#Transforming to csv creates an index column (Unnamed: 0) which need to be dropped
df_X_train.drop(columns=['Unnamed: 0'], inplace=True)
df_X_test.drop(columns=['Unnamed: 0'], inplace=True)
df_y_test.drop(columns=['Unnamed: 0'], inplace=True)
df_y_train.drop(columns=['Unnamed: 0'], inplace=True)

#Transforming the data
y_train = np.ravel(df_y_train)
y_test = np.ravel(df_y_test)

In [ ]:
#Count the values for each class 
df_y_train['Physics'].value_counts()
class_counts = [963, 874]  

In [ ]:
#Transforming the data
y_train = df_y_train[['Physics']].values.ravel() #Converting the dataframe type into a one-dimension array
y_test = df_y_test[['Physics']].values.ravel()

In [ ]:
#Scaling the data
scaler = StandardScaler()
scaler.fit(df_X_train)
X_train_scaler = scaler.transform(df_X_train)
X_test_scaler = scaler.transform(df_X_test)

In [ ]:
#Getting a first set of predictions to use for the evaluation metrics
#Selecting the number of neurons by the rule of thumb: 2/3 of inputs(X) + outputs(y) = 31
#A random state is applied so that th result will be consistent
mlp = MLPClassifier(random_state = 7)
mlp.fit(X_train_scaler,y_train)
y_pred = mlp.predict(X_test_scaler)
print(classification_report(y_test, y_pred,target_names=['FAIL','PASS']))

In [ ]:
#Using GridSearchCV to get finer results for the hyperparameters
#First GridSearch revolves around the rule-of-thumb value for the size of neurons in the hidden layer
mlp_gs = MLPClassifier()
hyperparams = {
    'hidden_layer_sizes' : [(50,),(75,),(90,),(100,)],
    'activation': ['logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.000001],
    'learning_rate_init': [0.00001,0.001],
    'max_iter' : [1000],
    'early_stopping' : [True],
    'random_state' : [7]
}

#Adjust the positive values to prioritize the outcome where the student fails
scorer = make_scorer(f1_score, pos_label = 0)

gs = GridSearchCV(
    estimator = mlp_gs,
    param_grid = hyperparams,
    cv = 5, 
    scoring = scorer,
    return_train_score = True
)

gs.fit(X_train_scaler,y_train)

In [ ]:
#Second GridSearch is used for a wider range and test if the rule-of-thumb is efficient for this case
mlp_gs = MLPClassifier()
hyperparams = {
    'hidden_layer_sizes' : [(10,),(15,),(20,),(25,),(30,)],
    'activation': ['logistic'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.000001],
    'learning_rate_init': [0.00001,0.001],
    'max_iter' : [1000],
    'early_stopping' : [True],
    'random_state' : [7]
}

#Adjust the positive values to prioritize the outcome where the student fails
scorer = make_scorer(f1_score, pos_label = 0)

gs = GridSearchCV(
    estimator = mlp_gs,
    param_grid = hyperparams,
    cv = 5, 
    scoring = scorer,
    return_train_score = True
)

gs.fit(X_train_scaler,y_train)

In [ ]:
res = pd.DataFrame(gs.cv_results_)
res_sorted = res.sort_values(by='rank_test_score',ascending=True)

In [ ]:
res_sorted_top = res_sorted[['param_activation','param_hidden_layer_sizes','param_learning_rate_init','param_solver','mean_test_score']].head(5)

In [ ]:
res_sorted_top['optimized_model_score'] = [0.75,0.74,0.72,0.73,0.59]
res_sorted_top['mean_test_score'] = res_sorted_top['mean_test_score'].round(3)
res_sorted_top

In [ ]:
#Creating a model with the optimized hyperparameters
mlp_optimized = MLPClassifier(activation = 'logistic',alpha= 0.000001,early_stopping= True,hidden_layer_sizes= (50,),learning_rate_init= 0.001,max_iter= 1000,random_state=7,solver= 'sgd')
mlp_optimized.fit(X_train_scaler,y_train)
y_pred = mlp_optimized.predict(X_test_scaler)
print(classification_report(y_test, y_pred,target_names=['FAIL','PASS']))

In [ ]:
#Using alternative method for comparison - KNeighbors
#This method is set to default values since this is not the technique being studied in this project.
kn = KNeighborsClassifier()
kn.fit(X_train_scaler,y_train)
y_pred_kn = kn.predict(X_test_scaler)
print(classification_report(y_test, y_pred_kn,target_names=['FAIL','PASS']))

In [ ]:
#GridSearchCV for KNeighbors
#Using GridSearchCV to get finer results for the hyperparameters
kn = KNeighborsClassifier()
param = {
    'n_neighbors' : [11,13,15,17,19,21,23,25,27,29,31]
}

#Adjust the positive values to prioritize the outcome where the student fails
scorer = make_scorer(f1_score, pos_label = 0)

gs_kn = GridSearchCV(
    estimator = kn,
    param_grid = param,
    cv = 5, 
    scoring = scorer,
    return_train_score = True
)

gs_kn.fit(X_train_scaler,y_train)

In [ ]:
res_kn = pd.DataFrame(gs_kn.cv_results_)
res_sorted_kn = res_kn.sort_values(by='rank_test_score',ascending=True)
res_sorted_kn = res_sorted_kn[['param_n_neighbors','mean_train_score']].head(5)
res_sorted_kn['optimized_model_score'] = [0.72,0.71,0.72,0.72,0.71]
res_sorted_kn['mean_train_score'] = res_sorted_kn['mean_train_score'].round(3)
res_sorted_kn

In [ ]:
#Optimized model for KNeighbors
kn = KNeighborsClassifier(n_neighbors = 27)
kn.fit(X_train_scaler,y_train)
y_pred_kn = kn.predict(X_test_scaler)
print(classification_report(y_test, y_pred_kn,target_names=['FAIL','PASS']))